In [2]:
# https://github.com/scienceai/neocortex/blob/master/examples/cifar10_cnn/cifar10_cnn_example.ipynb

import sys
sys.path.append('./modules/')
import tensorflow as tf

# from customLayers import conv_2d_BN, fully_connected_BN
from testing import testModel
from helpers import prepareRun 
import tflearn
from tflearn.layers import batch_normalization, local_response_normalization
from tflearn.data_utils import image_preloader

from tflearn.optimizers import momentum, Adam
from tflearn.activations import softmax, relu
from tflearn.layers.core import input_data, dropout
from tflearn.layers import max_pool_2d, regression, conv_2d, fully_connected
from tflearn.data_augmentation import * 
from tflearn.data_preprocessing import *
# from tflearn.data_utils import LabelPreloader, ImagePreloader, Preloader
tf.GraphKeys.LAYER_TENSOR = 'layer_tensor'

# https://www.kaggle.com/c/cifar-10/leaderboard?submissionId=3619756
# 81.750


hdf5 not supported (please install/reinstall h5py)
Scipy not supported!


In [5]:
struct = prepareRun('9-CNN-FracMaxPooling-b')

print(struct)

{'base': './models/9-CNN-FracMaxPooling-b/', 'run': '9-CNN-FracMaxPooling-b', 'Checkpoints': './models/9-CNN-FracMaxPooling-b/Checkpoints/', 'Tensorboard': './models/9-CNN-FracMaxPooling-b/Tensorboard/', 'Saves': './models/9-CNN-FracMaxPooling-b/Saves/'}


In [3]:
imgAug = ImageAugmentation()
imgAug.add_random_flip_leftright()
imgAug.add_random_flip_updown()
imgAug.add_random_crop((28, 28))

imgPrep = ImagePreprocessing()
imgPrep.add_crop_center((28, 28))
imgPrep.add_zca_whitening

# Model constructor
def buildModel():
    network = input_data(shape=[None, 28, 28, 3], data_augmentation=imgAug, data_preprocessing=imgPrep)
    # CNN Block 1
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 2
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 3
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 3
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # FC Block 1
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='relu')
    network = batch_normalization(network)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='Adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.05)
    return network
# Initialize the model and build the trainer:

net = buildModel()
model = tflearn.DNN(net, max_checkpoints=10, tensorboard_verbose=0,
                checkpoint_path=struct['Checkpoints'], tensorboard_dir=struct['Tensorboard'])

# Train the model - keep 10% (or 5000 images) as a validation set to gauge training effectiveness
model.fit(x, y, n_epoch=256, 
              validation_set=0.10, 
              shuffle=True, 
              show_metric=True, 
              batch_size=100,
              snapshot_step=1000, 
              snapshot_epoch=False, run_id=struct['run'])

model.save(struct['Saves'] + struct['run'])



Training Step: 115200  | total loss: 0.30789
| Adam | epoch: 256 | loss: 0.30789 - acc: 0.8994 -- iter: 45000/45000


In [4]:
results = testModel(model, './Data/test/*.png', struct['base'], struct['run'], save_to_csv=True)

Starting predictions
Predictions done, saving file.


In [6]:
struct['Saves'] + struct['run']
# 128 epochs @ 0.05
# https://www.kaggle.com/c/cifar-10/leaderboard?submissionId=3632127
# 80.750

'./models/9-CNN-FracMaxPooling-b/Saves/9-CNN-FracMaxPooling-b'

In [ ]:
# 4th layer:
# https://www.kaggle.com/c/cifar-10/leaderboard?submissionId=3636655
# 85.500

In [ ]:
# 4th layer w 10 pass eval:
# https://www.kaggle.com/c/cifar-10/leaderboard?submissionId=3638230
# 88.10

In [141]:
from PIL import Image
import numpy as np
import pandas as pd
from glob import glob
names = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [133]:
files = glob('./Ver/*.png')
proc = []

for f in files[:10]:
    files.append(f)
    i = Image.open(f)
    i = i.crop(box=[2, 2, 30, 30])
    i = np.array(i)  / 255.
    proc.append(i)
proc = np.array(proc)

In [134]:
r = model.predict(proc)

In [135]:
results = zip(r[:9], files)

In [137]:
files = glob('./Data/test/*.png')
names = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

results = []
for run in range(0, 10):
    n = 300000
    test_batch = 1000
    print('Starting predictions')
    for x in range(0, n / test_batch):
        images = files[x * test_batch: (x * test_batch) + test_batch]
        thisBatch = []
        for img in images:
            thisBatch.append(np.asarray(Image.open(img), dtype="float32") / 255.0)

        preds = model.predict(thisBatch)
        for i in range(0, test_batch):
            p = np.argmax(preds[i], 0) 
            results.append([run, images[i].split('/')[-1].split('.')[0], preds[i]])


Starting predictions
Starting predictions
Starting predictions
Starting predictions
Starting predictions
Starting predictions
Starting predictions
Starting predictions
Starting predictions
Starting predictions


In [143]:
df = pd.DataFrame(results)

In [236]:
for l in range(1, 10):
    temp = df[df[0] <= l]
    for c in range(0,10):
        temp['r' + str(c)] = temp[2].apply(lambda x: x[c])

    s = temp.groupby(1).agg({'r0':np.mean,'r1':np.mean,'r2':np.mean,'r3':np.mean,'r4':np.mean,
                       'r5':np.mean,'r6':np.mean,'r7':np.mean,'r8':np.mean, 'r9':np.mean})
    s = s.reset_index()
    s[1] = s[1].apply(lambda x: int(x))
    s = s[[1,'r0','r1','r2','r3','r4','r5','r6','r7','r8','r9']].sort_values(1)
    filenames = s[1]
    recs = np.array(s[['r0','r1','r2','r3','r4','r5','r6','r7','r8','r9']])
    recs = zip(filenames, np.argmax(recs, 1))
    final = [[r[0], names[r[1]]] for r in recs]
    pd.DataFrame(final).to_csv('multirun' + str(l + 1) + '.csv', index=None, header=['id','label'])

/home/mike/anaconda2/envs/tf11/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
